In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.stats import gaussian_kde

from dm4_functions import plot_dm4
from analysis_4DSTEM import fit_laue_circles, decompose_phi_from_circle, plot_laue_circle_results

In [12]:
data = np.load('4_Te_115x17_ss=3nm_CL=245_alpha=p48_p06sec_no beamstop_bin4_300kV_rotated_stack_float32.npy')
data_log = np.log(data - data.min() + 100)

peaks = np.load('locations_hough_nano4_day02_50_peaks_rot_blob_log.npy')

In [4]:
mask = np.loadtxt('nano4 recon mask.txt').astype(bool)
fig, ax = plt.subplots()
ax.matshow(mask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
box = 30
center = (245, 258) # approx

VBF = np.sum(data[..., center[0] - box//2:center[0] + box//2, center[1] - box//2 : center[1] + box//2], axis=(-2, -1))

# direct beam location
com = np.empty(data.shape[:2] + (2,))
for i, j in np.ndindex(data.shape[:2]):
    crop = data[i, j, center[0] - box//2:center[0] + box//2, center[1] - box//2 : center[1] + box//2]

    com[i, j] = ndimage.center_of_mass(crop)

kde = gaussian_kde((com[..., 0].ravel(), com[..., 1].ravel()), bw_method=2)

fig, ax = plt.subplots()
ax.matshow(VBF)

grid = np.meshgrid(np.linspace(com[..., 0].min(), com[..., 0].max()), np.linspace(com[..., 1].min(), com[..., 1].max()))

com_kde = kde(np.stack([i.ravel() for i in grid])).reshape(grid[0].shape).T
idx = np.unravel_index(com.argmax(), com_kde.shape)

cm0 = np.array(center) - box//2 + (grid[0][idx], grid[1][idx])
print(cm0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[244.41607062 258.45248215]


## Take 12 most intense spots

In [7]:
peaks12 = np.full(data.shape[:2] + (15, 2), np.nan)

for i, j in np.ndindex(data.shape[:2]):
    if not mask[i, j]:
        continue

    p = peaks[i, j][~np.isnan(peaks[i, j]).any(axis=1)][..., ::-1]

    intens = data[i, j][tuple(p.astype(int).T)]

    out = p[np.argsort(intens)[::-1]][:peaks12.shape[2]]
    peaks12[i, j][:out.shape[0]] = out

In [8]:
circle_parameters = np.load('circle_coords_fmin_nano4_day02_15_peaks_final.npy')

pixel_size = 0.04696889 # 1/nm
wavelength = 1.96e-3

phix, phiy = decompose_phi_from_circle(circle_parameters, cm0, pixel_size, wavelength)

In [9]:
fig, ax = plt.subplots(nrows=2)

layer = np.ones(data.shape[:2] + (4,))
layer[mask, -1] = 0

cmap= 'Spectral'
ax[0].matshow(phiy, cmap=cmap)
ax[1].matshow(phix, cmap=cmap)

for a in ax:
    a.imshow(layer)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plot_laue_circle_results(data_log, phix, phiy, VBF, mask, circle_parameters, peaks12)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=8, description='i', max=16), IntSlider(value=57, description='j', max=11…